In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
data_dir = Path('data')
full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')
)

In [3]:
full_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-03-01 00:31:13,2020-03-01 01:01:42,1.0,4.70,1.0,N,88,255,1,22.0,3.0,0.5,2.00,0.0,0.3,27.80,2.5,None
1,2,2020-03-01 00:08:22,2020-03-01 00:08:49,1.0,0.00,1.0,N,193,193,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80,0.0,None
2,1,2020-03-01 00:52:18,2020-03-01 00:59:16,1.0,1.10,1.0,N,246,90,1,6.0,3.0,0.5,1.95,0.0,0.3,11.75,2.5,None
3,2,2020-03-01 00:47:53,2020-03-01 00:50:57,2.0,0.87,1.0,N,151,238,1,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5,None
4,1,2020-03-01 00:43:19,2020-03-01 00:58:27,0.0,4.40,1.0,N,79,261,1,16.5,3.0,0.5,4.05,0.0,0.3,24.35,2.5,None


In [4]:
full_df.shape

(24649092, 19)

In [5]:
full_df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                      object
dtype: object

In [6]:
full_df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
count,2.464909e+07,2.383912e+07,2.464909e+07,2.383912e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.464909e+07,2.383912e+07
mean,1.682583e+00,1.467983e+00,3.527101e+00,1.048557e+00,1.639707e+02,1.611703e+02,1.238331e+00,1.266778e+01,1.071846e+00,5.127966e-01,2.082027e+00,3.038690e-01,2.975556e-01,1.842175e+01,2.271167e+00
std,5.033062e-01,1.112779e+00,3.250320e+02,7.610835e-01,6.675226e+01,7.095647e+01,5.282318e-01,2.740882e+02,1.007169e+02,1.007093e+02,2.610753e+00,1.604902e+00,3.745082e-02,3.402245e+02,7.473421e-01
min,1.000000e+00,0.000000e+00,-3.062000e+01,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.259000e+03,-2.700000e+01,-5.000000e-01,-4.932200e+02,-4.000000e+01,-3.000000e-01,-1.260300e+03,-2.500000e+00
25%,1.000000e+00,1.000000e+00,9.900000e-01,1.000000e+00,1.140000e+02,1.070000e+02,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,1.116000e+01,2.500000e+00
50%,2.000000e+00,1.000000e+00,1.650000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,1.920000e+00,0.000000e+00,3.000000e-01,1.430000e+01,2.500000e+00
75%,2.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,2.340000e+02,2.340000e+02,2.000000e+00,1.400000e+01,2.500000e+00,5.000000e-01,2.760000e+00,0.000000e+00,3.000000e-01,1.980000e+01,2.500000e+00
max,6.000000e+00,9.000000e+00,3.509149e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,9.983100e+05,5.000008e+05,5.000005e+05,1.393560e+03,9.255000e+02,3.000000e-01,1.000004e+06,3.000000e+00


In [ ]:
# profile = ProfileReport(full_df, title="Pandas Profiling Report")

In [ ]:
# profile.to_file("your_report.html")

In [ ]:
# profile.to_notebook_iframe()

In [7]:
full_df['VendorID'].value_counts()

2    16599044
1     8004823
6       45097
5         128
Name: VendorID, dtype: int64

## Regiones donde más suben y bajan pasajeros

In [8]:
df_pickup = pd.DataFrame(
    full_df['PULocationID'].value_counts().reset_index().values, columns=['LocationID', 'count_pickup'])

In [9]:
df_dropoff = pd.DataFrame(
    full_df['DOLocationID'].value_counts().reset_index().values, columns=['LocationID', 'count_df_dropoff'])

In [10]:
pickup_dropoff = pd.merge(df_pickup, df_dropoff, on='LocationID', how='inner')

In [11]:
pickup_dropoff['total'] = pickup_dropoff['count_pickup'] + pickup_dropoff['count_pickup']

In [12]:
df_geom_ny = pd.read_csv('data/taxi_zones.csv')

In [13]:
df_geom_ny

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island
...,...,...,...,...,...,...,...
258,256,0.067915,MULTIPOLYGON (((-73.95834207500002 40.71330630...,0.000169,Williamsburg (South Side),256,Brooklyn
259,259,0.126750,MULTIPOLYGON (((-73.85107116191898 40.91037152...,0.000395,Woodlawn/Wakefield,259,Bronx
260,260,0.133514,MULTIPOLYGON (((-73.90175373399988 40.76077547...,0.000422,Woodside,260,Queens
261,261,0.027120,MULTIPOLYGON (((-74.01332610899988 40.70503078...,0.000034,World Trade Center,261,Manhattan


In [14]:
pickup_dropoff = pd.merge(pickup_dropoff, df_geom_ny, on='LocationID', how='inner')

In [15]:
pickup_dropoff[[
    'LocationID', 'count_pickup', 'count_df_dropoff', 'total', 'zone', 'borough']]

,LocationID,count_pickup,count_df_dropoff,total,zone,borough
0,237,1145412,1015502,2290824,Upper East Side South,Manhattan
1,236,1089583,1126419,2179166,Upper East Side North,Manhattan
2,161,946854,843739,1893708,Midtown Center,Manhattan
3,186,862321,553455,1724642,Penn Station/Madison Sq West,Manhattan
4,162,831592,666196,1663184,Midtown East,Manhattan
...,...,...,...,...,...,...
254,176,29,236,58,Oakwood,Staten Island
255,111,26,318,52,Green-Wood Cemetery,Brooklyn
256,2,25,45,50,Jamaica Bay,Queens
257,30,20,164,40,Broad Channel,Queens


# Cuales son las horas y dias pico

In [16]:
full_df['pickup_hour'] = full_df['tpep_pickup_datetime'].map(lambda x: x.hour)
full_df['dropoff_hour'] = full_df['tpep_dropoff_datetime'].map(lambda x: x.hour)

In [31]:
df_pickup_hour = pd.DataFrame(
    full_df['pickup_hour'].value_counts().reset_index().values, columns=['hour', 'count_pickup'])

In [34]:
df_dropoff_hour = pd.DataFrame(
    full_df['dropoff_hour'].value_counts().reset_index().values, columns=['hour', 'count_dropoff'])

In [37]:
df_hour = pd.merge(df_pickup_hour, df_dropoff_hour, on='hour', how='inner')

In [39]:
df_hour.head()

,hour,count_pickup,count_dropoff
0,18,1688505,1722199
1,17,1641549,1615198
2,15,1605489,1601390
3,14,1578265,1536626
4,16,1530752,1532654
